# Bokeh: Interactive Web Visualizations in Python

## About Me

* PhD Candidate in the Interdisciplinary Quantitative Biology program and at the Institute for Behavioral Genetics at CU
* I work on addiction genomics and phenotyping methods that can scale to millions of subjects.
* <https://github.com/dbrazel>

## Why Bokeh?

* D3-like interactive web visualizations in Python, R, and Julia
    * "We write the JavaScript, so you don't have to!"
* Plays nicely with notebooks and produces portable, embeddable HTML
* Bokeh Server enables streaming updates

## Bokeh Architecture
![Bokeh Architecture](architecture.jpg)
(Peter Wang)

## Interfaces

* `bokeh.charts` - High-level, provides common statistical charts (bar, box, histogram, heat map, chord, etc.)
* `bokeh.plotting` - Mid-level, focused on placing glyphs (boxes, lines, circles, etc.)
* `bokeh.models` - Low-level, uses classes that map directly to BokehJS models

## Setup

In [25]:
import bokeh.charts as bkc
import bokeh.plotting as bkp

bkc.output_notebook()

# Load the seaborn exercise dataset

exercise = pd.read_csv('exercise.csv')
exercise = exercise.drop(exercise.columns[0], axis=1)
exercise.head()

Loading BokehJS ...

,id,diet,pulse,time,kind
0,1,low fat,85,1 min,rest
1,1,low fat,85,15 min,rest
2,1,low fat,88,30 min,rest
3,2,low fat,90,1 min,rest
4,2,low fat,92,15 min,rest


## `bokeh.charts`

In [28]:
p = bkc.BoxPlot(exercise, values='pulse', label='kind', color='kind', 
                title='Pulse in BPM, grouped by exercise type')
bkc.show(p)

It's easy to group by a column and to produce a self-contained HTML file that packages the required data.

In [41]:
#bkc.output_file('exercise_bar.html')
p = bkc.Bar(exercise, values='pulse', label='kind', color='time', agg='median', 
            group='time', title='Median BPM by kind of exercise, grouped by duration')
bkc.show(p)

In [42]:
from bokeh.sampledata.autompg import autompg

autompg.head()

#p = bkc.Scatter(autompg, x='hp', y='accel')
#bkc.show(p)

,mpg,cyl,displ,hp,weight,accel,yr,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
